In [11]:
import numpy as np
import pandas as pd
import json
from typing import List, Dict, Callable


In [12]:
with open('features.json') as f:
    feature_list = json.loads(f.read())
    features = {f['key']: f for f in feature_list}


In [13]:
raw_dataset = pd.read_pickle("./ad.pkl")
dataset = raw_dataset.copy()


In [14]:
dataset = dataset.dropna()

In [ ]:
dataset['DISTRICT'].value_counts()

In [16]:
def convert_numeric(feature: Dict) -> Callable[[str], int]:
    def convert(text: str):
        unit = feature.get('unit', "")
        if unit not in text:
            return None
        try:
            return np.int(text.replace(unit, "").replace(" ", ""))
        except ValueError:
            return None
    return convert


for key, feature in features.items():
    if feature['type'] == 'numeric':
        dataset[key] = dataset[key].map(
            convert_numeric(feature))


In [17]:
dataset = dataset.dropna()

In [18]:
dataset['PRICE'] = dataset['PRICE'].map(lambda p: p//1_000_000)


In [19]:
def validate_data(dataset: pd.DataFrame, feature: Dict) -> None:
    key, type = feature['key'], feature['type']

    def feature_filter_generator():
        if type == 'numeric':
            return lambda v: feature['min'] < v < feature['max']
        elif type == 'enum':
            return lambda v: v in feature['values']
    feature_filter = feature_filter_generator()
    return dataset[dataset[key].map(feature_filter)]

for feature in features.values():
    dataset = validate_data(dataset = dataset, feature = feature)

In [20]:
for key, feature in features.items():
    if feature['type'] == 'enum':
        dataset = pd.get_dummies(dataset, columns=[key], prefix=f'{key}_', prefix_sep='')

In [21]:
dataset.to_pickle('prepared_ad.pkl')